<a href="https://colab.research.google.com/github/elemi10/7506-TP-Org-de-datos/blob/7506-Trabajo-Practico-1/TP1_Real_or_Not%3F_NLP_with_Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **TP1 - Real or Not?**

###  **Análisis exploratorio**

![Banner_TP1.png](https://drive.google.com/uc?id=1BPA2RF1SDm9bTs1xZfVUa1VQn932E3wr)


In [0]:
#IMPORT FILES FROM DRIVE INTO GOOGLE-COLAB:

#STEP-1: Import Libraries

# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
#STEP-2: Autheticate E-Mail ID

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#STEP-3: Get File from Drive using file-ID

#2.1 Get the file
downloaded = drive.CreateFile({'id':'1RAGDjlzJ6spO5Sq8_x3UTIvxLhKAUBEt'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv') 

#downloaded = drive.CreateFile({'id':'17pAgG9oJRK1bAFWRKkp96__zicG6yUmy'}) # replace the id with id of file you want to access
#downloaded.GetContentFile('test.csv') 

#downloaded = drive.CreateFile({'id':'1u8v51BT7FZggIRD-eo0dQno--0wlxIhA'}) # replace the id with id of file you want to access
#downloaded.GetContentFile('sample_submission.csv') 


In [0]:
#STEP-4: Import Libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
import datetime as dt
import scipy as sp
#from sklearn.cross_validation import train_test_split -- no levanta
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,9)
plt.style.use('default') # mejoramos esteticamente un poco los gráficos en matplotlib
sns.set(style='whitegrid') # seteando tipo de grid en seaborn

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest
from sklearn import datasets, linear_model

### Levantamos archivo **train.csv**


In [29]:
df_train = pd.read_csv(r'train.csv', usecols=['keyword','location','text','target']) 
df_train.head(10)

,keyword,location,text,target
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,NaN,NaN,There's an emergency evacuation happening now ...,1
9,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [0]:
# exporto a excel
df_train.to_excel('df_train_preprocesado.xlsx')

In [31]:
df_train.info()
# las variables 'keyword' y 'location' estan incompletas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keyword   7552 non-null   object
 1   location  5080 non-null   object
 2   text      7613 non-null   object
 3   target    7613 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 238.0+ KB


In [0]:
#df_train['target'].value_counts()

### Creamos una variable para **contar total caracteres** del tweet

In [33]:
# Creo una columna con el cálculo de las palabras que tiene la columna 'text' y la llamo 'caracteres_tweet'. 
# Despues consulto con un head()
df_train['caracteres_tweet']=df_train['text'].str.len()
df_train.head(20)

,keyword,location,text,target,caracteres_tweet
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38
2,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133
3,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88
5,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,110
6,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,95
7,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,59
8,NaN,NaN,There's an emergency evacuation happening now ...,1,79
9,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,52


In [34]:
# Verificamos la existencia de valores nulos
df_train.isnull().any()

keyword              True
location             True
text                False
target              False
caracteres_tweet    False
dtype: bool

In [35]:
df_train.isnull().sum()

keyword               61
location            2533
text                   0
target                 0
caracteres_tweet       0
dtype: int64

In [36]:
df_train.describe()

,target,caracteres_tweet
count,7613.00000,7613.000000
mean,0.42966,101.037436
std,0.49506,33.781325
min,0.00000,7.000000
25%,0.00000,78.000000
50%,0.00000,107.000000
75%,1.00000,133.000000
max,1.00000,157.000000


In [37]:
# cuento total de tweets analizados
total_tweets=len(df_train)
total_tweets

7613

In [38]:
# verifico cuantos son verdaderos
tweets_verdaderos=(df_train['target']==1).sum()
tweets_verdaderos


3271

In [39]:
# verifico cuantos son fakes
tweets_falsos=(df_train['target']==0).sum()
tweets_falsos

4342

In [40]:
# calculo la probabilidad (casos favorables/sobre casos posibles) → en este casos es equivalente .mean()
probabilidad_tweet_verdadero = np.round((tweets_verdaderos)/(total_tweets),decimals=2)
probabilidad_tweet_verdadero

0.43

In [41]:
# calculo la probabilidad (casos favorables/sobre casos posibles) → en este casos es equivalente .mean()
probabilidad_tweet_falso = np.round((tweets_falsos)/(total_tweets),decimals=2)
probabilidad_tweet_falso

0.57

### **Iniciamos proceso de limpieza de datos**

#### **Verificación de nulos y normalización DF**



In [42]:
# Verifico que no haya instancias nulas o filas completas nulas
df_train=df_train.dropna(how="all")
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   keyword           7552 non-null   object
 1   location          5080 non-null   object
 2   text              7613 non-null   object
 3   target            7613 non-null   int64 
 4   caracteres_tweet  7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 356.9+ KB


In [43]:
# Normalizo el dataframe los registros NaN con texto
df_train=df_train.fillna({'keyword': 'sin keyword',\
                   'location': 'sin location'})
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   keyword           7613 non-null   object
 1   location          7613 non-null   object
 2   text              7613 non-null   object
 3   target            7613 non-null   int64 
 4   caracteres_tweet  7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 356.9+ KB


In [0]:
# Eliminamos los tweets duplicados considerando que su cantidad no tiene impacto en el análisis.
df_train=df_train.drop_duplicates('text')

In [45]:
df_train
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   keyword           7503 non-null   object
 1   location          7503 non-null   object
 2   text              7503 non-null   object
 3   target            7503 non-null   int64 
 4   caracteres_tweet  7503 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 351.7+ KB


## Preprocesamiento de datos: **Tokenización**

In [0]:
import string
punctuation = set(string.punctuation)

def tokenize(sentence):
    tokens = []
    for token in sentence.split():
        new_token = []
        for character in token:
            if character not in punctuation:
                new_token.append(character.lower())
        if new_token:
            tokens.append("".join(new_token))
    return tokens

In [0]:
# type(tokens)

In [0]:
df_train['text_vector']=df_train['text'].apply(tokenize)

In [49]:
df_train.head(5)

,keyword,location,text,target,caracteres_tweet,text_vector
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,69,"[our, deeds, are, the, reason, of, this, earth..."
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,38,"[forest, fire, near, la, ronge, sask, canada]"
2,sin keyword,sin location,All residents asked to 'shelter in place' are ...,1,133,"[all, residents, asked, to, shelter, in, place..."
3,sin keyword,sin location,"13,000 people receive #wildfires evacuation or...",1,65,"[13000, people, receive, wildfires, evacuation..."
4,sin keyword,sin location,Just got sent this photo from Ruby #Alaska as ...,1,88,"[just, got, sent, this, photo, from, ruby, ala..."


In [50]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 0 to 7612
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   keyword           7503 non-null   object
 1   location          7503 non-null   object
 2   text              7503 non-null   object
 3   target            7503 non-null   int64 
 4   caracteres_tweet  7503 non-null   int64 
 5   text_vector       7503 non-null   object
dtypes: int64(2), object(4)
memory usage: 410.3+ KB


In [0]:
#from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
#from sklearn.feature_extraction import stop_words
#print(stop_words.ENGLISH_STOP_WORDS)

In [52]:
import nltk
nltk.download('stopwords')

#from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [53]:
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
stopwords.fileids()

['arabic',
 'azerbaijani',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'slovene',
 'spanish',
 'swedish',
 'tajik',
 'turkish']

In [0]:
stop=stopwords.words('english')

In [0]:
def filtro(text_vector):
  text_vector_filtrado = []
  for palabra in text_vector:
    if palabra not in stop:
      text_vector_filtrado.append(palabra)
  return(text_vector_filtrado)

In [0]:
# type(text_vector_filtrado)

In [0]:
df_train['text_vector_filtrado']=df_train['text_vector'].apply(filtro)

In [61]:
df_train['text_vector_filtrado'].head()

0    [deeds, reason, earthquake, may, allah, forgiv...
1        [forest, fire, near, la, ronge, sask, canada]
2    [residents, asked, shelter, place, notified, o...
3    [13000, people, receive, wildfires, evacuation...
4    [got, sent, photo, ruby, alaska, smoke, wildfi...
Name: text_vector_filtrado, dtype: object

In [76]:
df_train['elem_vector']=df_train['text_vector'].str.len()
df_train.head()

,keyword,location,text,target,caracteres_tweet,text_vector,text_vector_filtrado,elem_vector,elem_vector_filtrado
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,69,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",13,7
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,38,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",7,7
2,sin keyword,sin location,All residents asked to 'shelter in place' are ...,1,133,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o...",22,11
3,sin keyword,sin location,"13,000 people receive #wildfires evacuation or...",1,65,"[13000, people, receive, wildfires, evacuation...","[13000, people, receive, wildfires, evacuation...",8,7
4,sin keyword,sin location,Just got sent this photo from Ruby #Alaska as ...,1,88,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi...",16,9


In [63]:
df_train['elem_vector_filtrado']=df_train['text_vector_filtrado'].str.len()
df_train.head()

,keyword,location,text,target,caracteres_tweet,text_vector,text_vector_filtrado,elem_vector,elem_vector_filtrado
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,69,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",13,7
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,38,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",7,7
2,sin keyword,sin location,All residents asked to 'shelter in place' are ...,1,133,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o...",22,11
3,sin keyword,sin location,"13,000 people receive #wildfires evacuation or...",1,65,"[13000, people, receive, wildfires, evacuation...","[13000, people, receive, wildfires, evacuation...",8,7
4,sin keyword,sin location,Just got sent this photo from Ruby #Alaska as ...,1,88,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi...",16,9


In [64]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 0 to 7612
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   keyword               7503 non-null   object
 1   location              7503 non-null   object
 2   text                  7503 non-null   object
 3   target                7503 non-null   int64 
 4   caracteres_tweet      7503 non-null   int64 
 5   text_vector           7503 non-null   object
 6   text_vector_filtrado  7503 non-null   object
 7   elem_vector           7503 non-null   int64 
 8   elem_vector_filtrado  7503 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 586.2+ KB


### Categorizamos la variable ***Keyword***

In [0]:
# "Limpiamos" la serie de caracteres especiales, sustituyendolos
df_train['keyword']=df_train['keyword'].str.replace('%20','_')

In [84]:
# Verificamos actualización
df_train['keyword'].unique()

array(['sin keyword', 'ablaze', 'accident', 'aftershock',
       'airplane_accident', 'ambulance', 'annihilated', 'annihilation',
       'apocalypse', 'armageddon', 'army', 'arson', 'arsonist', 'attack',
       'attacked', 'avalanche', 'battle', 'bioterror', 'bioterrorism',
       'blaze', 'blazing', 'bleeding', 'blew_up', 'blight', 'blizzard',
       'blood', 'bloody', 'blown_up', 'body_bag', 'body_bagging',
       'body_bags', 'bomb', 'bombed', 'bombing', 'bridge_collapse',
       'buildings_burning', 'buildings_on_fire', 'burned', 'burning',
       'burning_buildings', 'bush_fires', 'casualties', 'casualty',
       'catastrophe', 'catastrophic', 'chemical_emergency', 'cliff_fall',
       'collapse', 'collapsed', 'collide', 'collided', 'collision',
       'crash', 'crashed', 'crush', 'crushed', 'curfew', 'cyclone',
       'damage', 'danger', 'dead', 'death', 'deaths', 'debris', 'deluge',
       'deluged', 'demolish', 'demolished', 'demolition', 'derail',
       'derailed', 'derailmen